# 导入包和数据

In [1]:
import pandas
import numpy
import gurobipy as gp
import json

# read data
with open("sorted_ToyData_dict.json") as json_file:
     toy_data_dict= json.load(json_file)
with open("location.json") as json_file:
     location = json.load(json_file)
with open("slot.json") as json_file:
     slot = json.load(json_file)
with open("data_req.json") as json_file:
     demand = json.load(json_file)
with open("bandwidth.json") as json_file:
     bandwidth = json.load(json_file)

# 初始化常量

In [2]:
# number of jobs and slots
w = len(toy_data_dict)
J = len(slot)

# number of tasks in a job in stage 1
L = []
for key in toy_data_dict.keys():
    L.append(len(toy_data_dict[key]['stage']['1']))

# slots number in a data center
a = [];
for val in slot.values():
    a.append(val)

# all tasks in stage 1
tasks_1 = []
for i in toy_data_dict.keys():
    tasks_1.append(toy_data_dict[i]['stage']['1'])

# normalize location
for (key,val) in location.items():
    s = val[2:]
    location[key] = int(s)

# execution time of certain tasks
num = 0
e = []
for i in toy_data_dict.keys():
    exc = []
    for j in toy_data_dict[i]['Execution Time'].keys():
        if j in tasks_1[num]:
            exc.append(toy_data_dict[i]['Execution Time'][j])
    e.append(exc)
    num += 1
    
# communication time matrix
c = []
for k in range(w):
    com = []
    for i in range(len(tasks_1[k])):
        compute = []
        for j in range(J):
            maximum = 0
            for loc in demand[tasks_1[k][i]]:
                src = location[loc] - 1
                time = demand[tasks_1[k][i]][loc]/bandwidth[src][j]
                if(time > maximum):
                    maximum = time
            compute.append(maximum)
        com.append(compute)
    c.append(com)
    
# completion time records
complete_time = {}
job_complete_time = {}

# completed job records
complete_jobs = []

# record global time
global_time = 0

# record current stages and total stages
stages = [1,1,1,1,1,1]
total_stages = []
s = ''
for i in toy_data_dict.keys():
    for j in toy_data_dict[i]['stage'].keys():
        s = int(j)
    total_stages.append(s)
    
# slots occupied by jobs
occ_slots = {}

In [3]:
# set thresholds for rescheduling
time_threshold = 10
job_threshold = 6

# 线性求解接口实现

In [4]:
def LP(K, L, avai, M):
    # define model
    m = gp.Model()
    
    # add variables
    x = [0] * w
    X = [0] * w
    for k in K:
        x[k] = m.addVars(L[k],J,2,name="lambda"+str(k))
        X[k] = m.addVars(L[k],J,name="x"+str(k))
        
    # update model
    m.update()
    
    # set objective
    m.setObjective(
    gp.quicksum(x[k][i,j,0]+x[k][i,j,1]*pow(M,c[k][i][j]+e[k][i])
            for k in K for i in range(L[k]) for j in range(J)),
    gp.GRB.MINIMIZE
    )
    
    # add constraints
    m.addConstrs(X[k][i,j] == x[k][i,j,1] 
            for k in K for i in range(L[k]) for j in range(J))
    m.addConstrs(x[k][i,j,0] + x[k][i,j,1] == 1
            for k in K for i in range(L[k]) for j in range(J))
    m.addConstrs(gp.quicksum(X[k][i,j] for k in K for i in range(L[k]))
            <= avai[j] for j in range(J))
    m.addConstrs(gp.quicksum(X[k][i,j] for j in range(J)) == 1
            for k in K for i in range(L[k]))
    
    # solve it
    m.optimize()
    
    # res records the assigned tasks locations
    res = []
    for var in m.getVars():
        if(var.X == 1.0 and var.varName[0] == 'x'):
            res.append(int(var.varName[5:-1])+1)
    return res

In [5]:
def update(K, avai, M):
    num = 0
    maxim = 0
    bottleneck = 0
    
    # find the task with the longest time
    for i in K:
        for j in range(len(tasks_new[i])):
            time = c[i][j][res[num]-1] + e[i][j]
            if(time > maxim):
                maxim = time
                bottleneck = i
            num += 1
    complete_time[bottleneck] = global_time + maxim
    
    # find all task in the longest job
    occ_slots[bottleneck] = []
    setted = []
    num = 0
    for i in K:
        for j in range(len(tasks_new[i])):
            if(i == bottleneck):
                setted.append(res[num]-1)
                occ_slots[bottleneck].append(res[num]-1)
                location[tasks_new[i][j]] = res[num]
            num += 1
    
    # remove the longest job from the scheduling queue
    K.remove(bottleneck)
    M = 0
    
    # update M and available slots
    for i in K:
        M += J * L[i]
    for i in setted:
        avai[i] = avai[i] - 1
    return K, avai, M

# 多轮调度接口实现

In [6]:
def release_resource():
    # sort the jobs by their complete time
    complete_time_order = sorted(complete_time.items(), key=lambda x:x[1])
    
    # define help variables
    num = 0
    wait = 0
    start = complete_time_order[0][1]
    remained_jobs = w - len(job_complete_time)
    
    # update the completed jobs according to time and job thresholds
    complete_jobs.clear()
    for i in range(min(job_threshold,remained_jobs)):
        wait = (complete_time_order[i][1] - start)
        if(wait > time_threshold):
            global_time = start + time_threshold
            break
        global_time = complete_time_order[i][1]
        complete_jobs.append(complete_time_order[i][0])
        
    # find completed jobs and release the slots that have been occupied by the completed tasks
    num = 0
    remove = []
    for i in complete_jobs:
        if(stages[i] == total_stages[i]):
            remove.append(i)
            job_complete_time[i] = complete_time_order[num][1]
            del complete_time[i]
        stages[i] += 1
        for j in occ_slots[i]:
            avai[j] += 1
        occ_slots[i].clear()
        num += 1
        
    for i in remove:
        complete_jobs.remove(i)
        
    return global_time, complete_jobs, stages, job_complete_time, avai, occ_slots

In [7]:
def set_next_iteration():
    # initialize variables
    tasks_new.clear()
    K = complete_jobs.copy()
    num = 0
    
    # add new stages of completed jobs to the scheduling pool
    for i in toy_data_dict.keys():
        L[num] = 0
        if num in complete_jobs:
            tasks_new.append(toy_data_dict[i]['stage'][str(stages[num])])
            L[num] = len(toy_data_dict[i]['stage'][str(stages[num])])
        else:
            tasks_new.append([])
        num += 1
        
    # reset M for new tasks to be scheduled
    M = 0
    for i in K:
        M += J * L[i]
    
    # reset e for new tasks 
    e.clear()
    num = 0
    for i in toy_data_dict.keys():
        exc = []
        for j in toy_data_dict[i]['Execution Time'].keys():
            if j in tasks_new[num]:
                exc.append(toy_data_dict[i]['Execution Time'][j])
        e.append(exc)
        num += 1
        
    # reset c for new tasks
    c.clear()
    for k in range(w):
        com = []
        for i in range(len(tasks_new[k])):
            compute = []
            for j in range(J):
                maximum = 0
                for loc in demand[tasks_new[k][i]]:
                    src = location[loc] - 1
                    time = demand[tasks_new[k][i]][loc]/bandwidth[src][j]
                    if(time > maximum):
                        maximum = time
                compute.append(maximum)
            com.append(compute)
        c.append(com)
        
    return K, tasks_new, L, M, e, c

# 全过程调度

In [8]:
# 第一轮数据初始化
K = []
for i in range(w):
    K.append(i)

avai = a.copy()

M = 0
for i in K:
    M += J*L[i]
    
tasks_new = tasks_1.copy()

In [9]:
# 第一轮迭代
for i in range(w):
    res = LP(K,L,avai,M)
    K, avai, M = update(K, avai, M)

Academic license - for non-commercial use only - expires 2021-07-25
Using license file /Users/emiyali/gurobi.lic
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 229 rows, 312 columns and 624 nonzeros
Model fingerprint: 0x29185630
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 4e+11]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 223 rows and 299 columns
Presolve time: 0.01s
Presolved: 6 rows, 13 columns, 22 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9496924e+06   3.000000e+00   0.000000e+00      0s
       3    1.0981762e+09   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds
Optimal objective  1.098176167e+09
Gurobi Optimizer versi

In [10]:
# 全过程
while(len(complete_time) != 0):
    global_time, complete_jobs, stages, job_complete_time, avai, occ_slots = release_resource()
    K, tasks_new, L, M, e, c = set_next_iteration()
    for i in range(len(complete_jobs)):
        res = LP(K,L,avai,M)
        K, avai, M = update(K, avai, M)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 256 rows, 351 columns and 702 nonzeros
Model fingerprint: 0x92144942
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 9e+08]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 256 rows and 351 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.6173564e+06   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  8.617356384e+06
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 175 rows, 234 columns and 468 nonzeros
Model fingerprint: 0xd40a4769
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+08

  RHS range        [1e+00, 1e+01]
Presolve removed 40 rows and 39 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.4315686e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  3.431568625e+04


# 结果一览

In [11]:
job_complete_time

{0: 1.9411764705882353,
 1: 6.1491228070175445,
 3: 9.072512425529114,
 2: 9.638639281129654,
 4: 13.163275339360338,
 5: 17.23470391078891}

In [12]:
location

{'A1': 1,
 'A2': 5,
 'A3': 12,
 'A4': 1,
 'A5': 7,
 'A6': 4,
 'A7': 7,
 'B1': 6,
 'B2': 4,
 'B3': 5,
 'B4': 8,
 'B5': 4,
 'B6': 3,
 'B7': 13,
 'B8': 9,
 'C1': 10,
 'C2': 1,
 'C3': 5,
 'C4': 7,
 'C5': 1,
 'C6': 4,
 'D1': 6,
 'D2': 13,
 'D3': 10,
 'D4': 12,
 'D5': 2,
 'D6': 4,
 'D7': 3,
 'E1': 6,
 'E2': 11,
 'E3': 8,
 'E4': 8,
 'E5': 8,
 'E6': 5,
 'E7': 3,
 'F1': 7,
 'F2': 12,
 'F3': 5,
 'F4': 13,
 'F5': 4,
 'F6': 11,
 'F7': 5,
 'F8': 11,
 'tE1': 8,
 'tF1': 7,
 'tC1': 3,
 'tB1': 6,
 'tD1': 3,
 'tD2': 9,
 'tA1': 8,
 'tA2': 8,
 'tF2': 12,
 'tF3': 8,
 'tF4': 7,
 'tE2': 8,
 'tE3': 8,
 'tD3': 9,
 'tD4': 9,
 'tB2': 6,
 'tC2': 1,
 'tE4': 4,
 'tE5': 8,
 'tC3': 3,
 'tF5': 8,
 'tF6': 7,
 'tD5': 9,
 'tE6': 8,
 'tF7': 8,
 'tF8': 8,
 'tF9': 4}